## Final Capstone Project 

In [1]:
#import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

from bs4 import BeautifulSoup
import xml

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported and loaded.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported and loaded.


In [2]:
#This is the file about towns profile
link1 = 'https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(link1,encoding='latin1')
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [ ]:
#this is a list of towns
towns = list(df.columns.values)
towns = towns[5:]

In [4]:
#this is how to transform the used data into a dataframe
city_towns = pd.DataFrame(index=towns, columns=["population", "male", "female",'higher_education', "after_tax_income"])

for index, row in city_towns.iterrows():
    city_towns.at[index, 'population'] = df[index][2]
    city_towns.at[index, 'm2'] = df[index][20]
    city_towns.at[index, 'm3'] = df[index][21]
    city_towns.at[index, 'm4'] = df[index][22]
    city_towns.at[index, 'f2'] = df[index][41]
    city_towns.at[index, 'f3'] = df[index][42]
    city_towns.at[index, 'f4'] = df[index][43]
    city_towns.at[index, 'higher_education'] = df[index][1723]
    city_towns.at[index, 'after_tax_income'] = df[index][2354]
city_towns.reset_index(inplace=True)
city_towns.head()

,index,population,male,female,higher_education,after_tax_income,m2,m3,m4,f2,f3,f4
0,Agincourt North,"29,113",NaN,NaN,4240,"26,955",1015,835,680,1005,935,775
1,Agincourt South-Malvern West,"23,757",NaN,NaN,4615,"27,928",1045,820,625,975,835,715
2,Alderwood,"12,054",NaN,NaN,1980,"39,159",355,410,455,350,430,450
3,Annex,"30,526",NaN,NaN,12640,"80,138",2080,1610,1055,2265,1675,1040
4,Banbury-Don Mills,"27,695",NaN,NaN,8060,"51,874",645,735,735,745,860,895


In [5]:
city_towns.columns = city_towns.columns.str.replace('index', 'neighborhood')
city_towns['population'] = city_towns['population'].str.replace(',','').astype(int)
city_towns['male'] = (city_towns['m2'].astype(int) + city_towns['m3'].astype(int) + city_towns['m4'].astype(int))
city_towns['female'] = (city_towns['f2'].astype(int) + city_towns['f3'].astype(int) + city_towns['f4'].astype(int))
city_towns['higher_education'] = city_towns['higher_education'].astype(int)
city_towns['after_tax_income'] = city_towns['after_tax_income'].str.replace(',','').astype(int)
city_towns.head()

,neighborhood,population,male,female,higher_education,after_tax_income,m2,m3,m4,f2,f3,f4
0,Agincourt North,29113,2530,2715,4240,26955,1015,835,680,1005,935,775
1,Agincourt South-Malvern West,23757,2490,2525,4615,27928,1045,820,625,975,835,715
2,Alderwood,12054,1220,1230,1980,39159,355,410,455,350,430,450
3,Annex,30526,4745,4980,12640,80138,2080,1610,1055,2265,1675,1040
4,Banbury-Don Mills,27695,2115,2500,8060,51874,645,735,735,745,860,895


In [7]:
# this is for scrapping data
url = requests.get('https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto').text
soup = BeautifulSoup(url,'lxml')

In [8]:
table_post = soup.find('table')
fields = table_post.find_all('td')

CDN_number = []
city_designated_area = []
former_city_borough = []


for i in range(0, len(fields), 5):
    CDN_number.append(fields[i].text.strip())
    city_designated_area.append(fields[i+1].text.strip())
    former_city_borough.append(fields[i+2].text.strip())
            
df5 = pd.DataFrame(data=[CDN_number, city_designated_area, former_city_borough]).transpose()
df5.columns = ['CDN', 'neighborhood', 'borough']
df5 = df5.drop(['CDN'], axis=1)
df5.reset_index()
df5.head()

,neighborhood,borough
0,Agincourt North,Scarborough
1,Agincourt South-Malvern West,Scarborough
2,Alderwood,Etobicoke
3,Annex,Old City of Toronto
4,Banbury-Don Mills,North York


In [12]:
#create appropriate and clean data frames
link1 = 'https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=csv&projection=4326'
df2 = pd.read_csv(link1,encoding='latin1')
df3 = df2.filter(['AREA_NAME','AREA_DESC','LONGITUDE','LATITUDE'], axis=1)
df3['AREA_NAME'] = df3['AREA_NAME'].str.replace('\d+','').str.replace("(","").str.replace(')', '')
df3.columns = map(str.lower, df3.columns)
df3.columns = df3.columns.str.replace('area_name', 'neighborhood')
df3 = df3.sort_values(by=['neighborhood'])
df3 = df3.reset_index()
df3 = df3.drop(['index'], axis=1)
df4 = pd.merge(df3, df_xy, left_index=True, right_index=True, how='inner')
df4 = df4.drop(['neighborhood_y'], axis=1)
df4.columns = df4.columns.str.replace('neighborhood_x', 'neighborhood')
df4 = df4[['borough','neighborhood','area_desc', 'longitude','latitude','population', 'male', 'female', 'higher_education', 'after_tax_income']]
df4 = df4.drop(['neighborhood'], axis=1)
df4.columns = df4.columns.str.replace('area_desc', 'neighborhood') 
df4.head()

,borough,neighborhood,longitude,latitude,population,male,female,higher_education,after_tax_income
0,Scarborough,Agincourt North (129),-79.266712,43.805441,29113,2530,2715,4240,26955
1,Scarborough,Agincourt South-Malvern West (128),-79.265612,43.788658,23757,2490,2525,4615,27928
2,Etobicoke,Alderwood (20),-79.541611,43.604937,12054,1220,1230,1980,39159
3,Old City of Toronto,Annex (95),-79.404001,43.671585,30526,4745,4980,12640,80138
4,North York,Banbury-Don Mills (42),-79.349718,43.737657,27695,2115,2500,8060,51874


In [ ]:
#this is to make some useful median operations
pop_med = df4['population'].median()
male_med = df4['male'].median()
female_med = df4['female'].median()
edu_med = df4['higher_education'].median()
income_med = df4['after_tax_income'].median()

In [14]:
#also producecorresponding scores
df_score = pd.DataFrame(columns=["neighborhood","pop_score", "male_score", "female_score", "edu_score", "income_score", "total_score"])
df_score['neighborhood'] = df4['neighborhood']

pop_score = [] # 15% importance
for x in df4['population']:
  if x / pop_med >0:
    pop_score.append((x / pop_med)*.15)
  else:
    pop_score.append(0)
df_score['pop_score'] = pop_score

male_score = [] # 25% importance
for z in df4['male']:
  if z / male_med >0:
    male_score.append((z / male_med)*.25)
  else:
    male_score.append(0)
df_score['male_score'] = male_score

female_score = [] # 25% importance
for z in df4['female']:
  if z / female_med >0:
    female_score.append((z / female_med)*.25)
  else:
    female_score.append(0)
df_score['female_score'] = female_score

edu_score = [] # 15% importance
for z in df4['higher_education']:
  if z / edu_med >0:
    edu_score.append((z / edu_med)*.15)
  else:
    edu_score.append(0)
df_score['edu_score'] = edu_score

income_score = [] # 20% importance
for z in df4['after_tax_income']:
  if z / income_med >0:
    income_score.append((z / income_med)*.2)
  else:
    income_score.append(0)
df_score['income_score'] = income_score
df_score['total_score'] = round(df_score.iloc[:,-7:].sum(axis=1),2)

df_score.head()

,neighborhood,pop_score,male_score,female_score,edu_score,income_score,total_score
0,Agincourt North (129),0.260721,0.351389,0.347631,0.154275,0.147543,1.26
1,Agincourt South-Malvern West (128),0.212756,0.345833,0.323303,0.167920,0.152869,1.20
2,Alderwood (20),0.107949,0.169444,0.157490,0.072044,0.214344,0.72
3,Annex (95),0.273375,0.659028,0.637644,0.459915,0.438650,2.47
4,Banbury-Don Mills (42),0.248022,0.293750,0.320102,0.293269,0.283942,1.44


In [ ]:
#Merging and cleaning
df6 = pd.merge(df4, df_score, left_index=True, right_index=True, how='inner')
df6 = df6.drop(['neighborhood_y'], axis=1)
df6.columns = df6.columns.str.replace('neighborhood_x', 'neighborhood')
df6 = df6.drop(columns = ['population','male','female','higher_education','after_tax_income'])

In [16]:
!wget --quiet https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=geojson&projection=4326 
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.
'projection' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
toronto_geo = r'Neighbourhoods.geojson'

address = 'Toronto'

geolocator = Nominatim(user_agent="torcan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5, tiles = 'Mapbox Bright')

In [40]:
#this is to use foursquare
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [21]:
#top venus within a specific area
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius=1610): # 1610 meters = 1.00 mile
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood_latitude', 
                  'neighborhood_longitude', 
                  'venue', 
                  'venue_latitude', 
                  'venue_longitude', 
                  'venue_category']
    
    return(nearby_venues)

In [22]:
#obtaining the results
tor_neigh_geo = df6
tor_neigh_venues = getNearbyVenues(names=tor_neigh_geo['neighborhood'],
                                  latitudes=tor_neigh_geo['latitude'],
                                  longitudes=tor_neigh_geo['longitude'],
                                  LIMIT=100)

print('The "tor_neigh_venues" dataframe has {} venues and {} unique venue types.'.format(
      len(tor_neigh_venues['venue_category']),
      len(tor_neigh_venues['venue_category'].unique())))

tor_neigh_venues.head()

Agincourt North (129)
Agincourt South-Malvern West (128)
Alderwood (20)
Annex (95)
Banbury-Don Mills (42)
Bathurst Manor (34)
Bay Street Corridor (76)
Bayview Village (52)
Bayview Woods-Steeles (49)
Bedford Park-Nortown (39)
Beechborough-Greenbrook (112)
Bendale (127)
Birchcliffe-Cliffside (122)
Black Creek (24)
Blake-Jones (69)
Briar Hill-Belgravia (108)
Bridle Path-Sunnybrook-York Mills (41)
Broadview North (57)
Brookhaven-Amesbury (30)
Cabbagetown-South St.James Town (71)
Caledonia-Fairbank (109)
Casa Loma (96)
Centennial Scarborough (133)
Church-Yonge Corridor (75)
Clairlea-Birchmount (120)
Clanton Park (33)
Cliffcrest (123)
Corso Italia-Davenport (92)
Danforth (66)
Danforth East York (59)
Don Valley Village (47)
Dorset Park (126)
Dovercourt-Wallace Emerson-Junction (93)
Downsview-Roding-CFB (26)
Dufferin Grove (83)
East End-Danforth (62)
Edenbridge-Humber Valley (9)
Eglinton East (138)
Elms-Old Rexdale (5)
Englemount-Lawrence (32)
Eringate-Centennial-West Deane (11)
Etobicoke West

,neighborhood,neighborhood_latitude,neighborhood_longitude,venue,venue_latitude,venue_longitude,venue_category
0,Agincourt North (129),43.805441,-79.266712,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
1,Agincourt North (129),43.805441,-79.266712,Fahmee Bakery & Jamaican Foods,43.810170,-79.280113,Caribbean Restaurant
2,Agincourt North (129),43.805441,-79.266712,Samosa King - Embassy Restaurant,43.810152,-79.257316,Indian Restaurant
3,Agincourt North (129),43.805441,-79.266712,Grandeur Palace 華丽宮 (Grandeur Palace 華麗宮),43.797885,-79.270585,Chinese Restaurant
4,Agincourt North (129),43.805441,-79.266712,Bestco Food Market 鴻華超級市場,43.796514,-79.270790,Supermarket


In [27]:
# creating a map with geographic locations
high_towns_visual = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, venue, category in zip(top15_hoods_venues['venue_latitude'], top15_hoods_venues['venue_longitude'], top15_hoods_venues['venue'], top15_hoods_venues['venue_category']):
        label = '{}, {}'.format(category, venue)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=0.1,
            popup=label,
            color='red',
            fill=True,
            fill_color='#FF0000',
            fill_opacity=0.3).add_to(high_towns_visual)  

high_towns_visual

In [30]:
#grouping
high_towns = pd.get_dummies(top15_hoods_venues[['venue_category']], prefix="", prefix_sep="")
high_towns['neighborhood'] = top15_hoods_venues['neighborhood'] 
fixed_columns = [high_towns.columns[-1]] + list(high_towns.columns[:-1])
high_towns = high_towns[fixed_columns]
high_towns.drop(['Neighborhood'], axis=1, inplace=True)
high_all_towns = high_towns.groupby('neighborhood').sum().reset_index()
high_all_towns.columns = map(str.lower, high_all_towns.columns)
high_all_towns

,neighborhood,airport,american restaurant,antique shop,aquarium,arcade,art gallery,art museum,arts & crafts store,asian restaurant,athletics & sports,bbq joint,baby store,bagel shop,bakery,bank,bar,baseball field,baseball stadium,basketball stadium,beach,beer bar,beer store,big box store,bike shop,bistro,bookstore,boutique,brazilian restaurant,breakfast spot,brewery,bubble tea shop,burger joint,burrito place,bus line,bus station,butcher,café,caribbean restaurant,castle,cheese shop,chinese restaurant,circus,clothing store,cocktail bar,coffee shop,comedy club,comfort food restaurant,comic shop,concert hall,convenience store,cosmetics shop,coworking space,creperie,cuban restaurant,cupcake shop,dance studio,deli / bodega,department store,design studio,dessert shop,diner,discount store,dive bar,dog run,donut shop,dumpling restaurant,eastern european restaurant,electronics store,ethiopian restaurant,event space,falafel restaurant,fast food restaurant,fish & chips shop,fish market,food,food & drink shop,food truck,french restaurant,fried chicken joint,furniture / home store,garden,gastropub,gay bar,general entertainment,german restaurant,gift shop,golf course,gourmet shop,government building,greek restaurant,grocery store,gym,gym / fitness center,hakka restaurant,harbor / marina,hardware store,health food store,historic site,history museum,hobby shop,hockey arena,hotel,hotel bar,ice cream shop,indian restaurant,indie movie theater,indonesian restaurant,intersection,italian restaurant,japanese restaurant,jewelry store,jewish restaurant,juice bar,korean restaurant,lake,latin american restaurant,liquor store,lounge,mediterranean restaurant,men's store,metro station,mexican restaurant,middle eastern restaurant,miscellaneous shop,modern european restaurant,monument / landmark,movie theater,museum,music school,music store,music venue,nail salon,new american restaurant,office,opera house,optical shop,other great outdoors,paper / office supplies store,park,pastry shop,performing arts venue,persian restaurant,pet store,pharmacy,pizza place,plaza,poke place,pool,pool hall,portuguese restaurant,pub,racetrack,ramen restaurant,record shop,restaurant,rock club,salad place,salon / barbershop,sandwich place,scenic lookout,school,seafood restaurant,shopping mall,skating rink,smoke shop,snack place,soccer field,soccer stadium,soup place,south american restaurant,spa,spanish restaurant,speakeasy,sporting goods shop,sports bar,stationery store,steakhouse,supermarket,sushi restaurant,taco place,tapas restaurant,tea room,tennis court,thai restaurant,theater,theme park,theme restaurant,thrift / vintage store,toy / game store,track,trail,train station,tram station,turkish restaurant,udon restaurant,university,vegetarian / vegan restaurant,video game store,vietnamese restaurant,warehouse store,wine bar,wings joint,women's store,yoga studio,zoo,zoo exhibit
0,Annex (95),0,1,0,0,0,0,0,1,0,0,2,0,0,3,0,1,0,0,0,0,2,0,0,1,0,1,2,0,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0,5,0,0,0,1,0,1,0,0,0,0,0,2,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,2,0,1,0,1,2,1,0,0,0,0,1,1,1,0,0,2,0,2,2,1,0,0,4,3,1,1,1,1,0,1,0,1,2,1,0,2,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,3,0,0,0,1,0,1,0,0,0,3,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0
1,Bay Street Corridor (76),0,2,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,2,0,1,1,2,0,0,0,4,0,0,0,0,0,1,0,6,0,0,1,1,0,2,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,5,1,0,0,0,0,0,0,1,0,2,1,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,2,3,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,3,0,0,0,0,0,3,2,0,0,0,0,1,0,3,1,2,0,0,2,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,0,2,2,0,1,0,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,1,0,0
2,Church-Yonge Corridor (75),0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,1,1,0,0,0,2,0,0,0,0,0,1,0,4,0,0,1,1,0,2,0,2,0,0,2,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,6,2,0,1,0,0,0,0,0,0,2,0,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,4,4,0,0,1,0,0,1

In [32]:
high_sum_towns = pd.DataFrame(columns=["neighborhood","total"])
high_sum_towns['neighborhood'] = high_all_towns['neighborhood']
high_sum_towns['total'] = high_all_towns.iloc[:,-111:].sum(axis=1)
high_sum_towns

,neighborhood,total
0,Annex (95),57
1,Bay Street Corridor (76),59
2,Church-Yonge Corridor (75),62
3,Dovercourt-Wallace Emerson-Junction (93),59
4,Islington-City Centre West (14),36
5,Lambton Baby Point (114),34
6,Malvern (132),10
7,Mimico (includes Humber Bay Shores) (17),51
8,Mount Pleasant West (104),53
9,Niagara (82),56


In [34]:
# clustering using k-means
clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
final_hoods_clustering = final_hoods_grouped.drop('neighborhood', 1) # neighborhood in this dataset is a categorical variable
k_means.fit(final_hoods_clustering)
label = k_means.labels_
print(label)

[2 2 2 0 1 1 1 1 2 0 0 0 1 1]


In [35]:
last_outcome = pd.DataFrame(k_means.cluster_centers_)
last_outcome.columns = final_hoods_clustering.columns
last_outcome.index = ['cluster_0','cluster_1', 'cluster_2']
last_outcome['total'] = last_outcome.sum(axis = 1)
last_outcome

,american restaurant,art gallery,art museum,asian restaurant,bbq joint,bar,baseball stadium,basketball stadium,beach,beer bar,beer store,bistro,brazilian restaurant,brewery,burger joint,burrito place,caribbean restaurant,chinese restaurant,cocktail bar,comedy club,comfort food restaurant,concert hall,coworking space,cuban restaurant,diner,dive bar,dumpling restaurant,eastern european restaurant,ethiopian restaurant,event space,falafel restaurant,fast food restaurant,fish & chips shop,food & drink shop,food truck,french restaurant,fried chicken joint,gastropub,gay bar,general entertainment,german restaurant,golf course,greek restaurant,grocery store,hakka restaurant,history museum,hockey arena,hotel,hotel bar,indian restaurant,indie movie theater,indonesian restaurant,italian restaurant,japanese restaurant,jewish restaurant,korean restaurant,lake,latin american restaurant,lounge,mediterranean restaurant,mexican restaurant,middle eastern restaurant,modern european restaurant,monument / landmark,movie theater,museum,music venue,new american restaurant,office,opera house,park,performing arts venue,persian restaurant,pizza place,poke place,pool,pool hall,portuguese restaurant,pub,racetrack,ramen restaurant,restaurant,rock club,salad place,sandwich place,seafood restaurant,skating rink,soccer field,soccer stadium,soup place,south american restaurant,spanish restaurant,speakeasy,sports bar,steakhouse,sushi restaurant,taco place,tapas restaurant,thai restaurant,theater,theme park,theme restaurant,trail,turkish restaurant,udon restaurant,university,vegetarian / vegan restaurant,vietnamese restaurant,wine bar,wings joint,total
cluster_0,1.500000,7.500000e-01,0.000000e+00,5.000000e-01,0.250000,4.250000,5.000000e-01,2.500000e-01,5.000000e-01,1.00,0.250000,2.500000e-01,2.500000e-01,2.750000,0.500000,0.00,0.000000,0.500000,1.5,5.000000e-01,0.500000,0.50,2.500000e-01,2.500000e-01,0.750000,2.500000e-01,2.500000e-01,0.000000,5.000000e-01,2.500000e-01,2.500000e-01,0.000000,0.0,0.000000e+00,0.000000e+00,0.500000,0.5,0.750000,0.00,0.000000,0.000000e+00,0.000000,0.500000,0.750000,0.000000,0.50,2.500000e-01,2.500000,0.75,0.500000,0.000000,0.000000,4.750000,0.500000,0.000000e+00,0.250000,5.000000e-01,0.500000,2.500000e-01,0.250000,0.500000,0.750000,0.000000e+00,2.500000e-01,0.250000,0.25,0.75,0.75,2.500000e-01,0.000000e+00,4.000000,5.000000e-01,0.000000e+00,1.250000,2.500000e-01,0.000000,0.000000e+00,2.500000e-01,1.250000,5.000000e-01,0.25,1.750000,2.500000e-01,2.500000e-01,1.000000,0.500000,0.250000,2.500000e-01,5.000000e-01,2.500000e-01,2.500000e-01,0.000000,0.000000e+00,0.250000,0.500000,1.250000,0.250000,0.000000e+00,1.0,0.250000,2.500000e-01,0.000000e+00,0.25,0.000000,0.000000,0.000000e+00,1.0,1.0,0.50,0.000000,58.000000
cluster_1,0.666667,5.551115e-17,2.775558e-17,5.551115e-17,0.333333,0.666667,2.775558e-17,1.387779e-17,2.775558e-17,0.00,0.833333,1.387779e-17,2.775558e-17,0.333333,1.333333,0.50,0.166667,0.666667,0.0,2.775558e-17,0.333333,0.00,1.387779e-17,1.387779e-17,0.666667,1.387779e-17,1.387779e-17,0.166667,2.775558e-17,1.387779e-17,1.387779e-17,2.333333,0.5,1.387779e-17,1.387779e-17,0.333333,1.0,0.166667,0.00,0.166667,1.387779e-17,0.166667,0.333333,2.166667,0.166667,0.00,1.387779e-17,0.166667,0.00,0.833333,0.166667,0.166667,1.166667,1.166667,1.387779e-17,1.833333,2.775558e-17,0.166667,2.775558e-17,0.166667,0.166667,0.166667,2.775558e-17,2.775558e-17,0.333333,0.00,0.00,0.00,1.387779e-17,1.387779e-17,1.833333,2.775558e-17,1.387779e-17,2.833333,1.387779e-17,0.166667,1.387779e-17,2.775558e-17,0.833333,2.775558e-17,0.50,2.166667,1.387779e-17,1.387779e-17,1.666667,0.333333,0.166667,1.387779e-17,2.775558e-17,1.387779e-17,1.387779e-17,0.166667,1.387779e-17,0.166667,0.166667,1.833333,0.166667,1.387779e-17,1.0,0.333333,1.387779e-17,2.775558e-17,0.00,0.333333,0.166667,1.387779e-17,0.0,0.5,0.00,0.333333,36.166667
cluster_2,1.000000,2.500000e-01,5.000000e-01,5.000000e-01,0.750000,0.500000,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000e+

In [36]:
last_outcome_all = pd.DataFrame(final_hoods_grouped['neighborhood'])
last_outcome_all = last_outcome_all.assign(cluster_labels = label)

In [37]:
print(last_outcome_all.shape)
last_outcome_all

(14, 2)


,neighborhood,cluster_labels
0,Annex (95),2
1,Bay Street Corridor (76),2
2,Church-Yonge Corridor (75),2
3,Dovercourt-Wallace Emerson-Junction (93),0
4,Islington-City Centre West (14),1
5,Lambton Baby Point (114),1
6,Malvern (132),1
7,Mimico (includes Humber Bay Shores) (17),1
8,Mount Pleasant West (104),2
9,Niagara (82),0


In [38]:
#show the remaining appropriate towns
remaining_towns = top15_hoods
remaining_towns = remaining_towns.join(last_outcome_all.set_index('neighborhood'), on='neighborhood')
remaining_towns = remaining_towns.drop(['pop_score', 'male_score', 'female_score', 'edu_score', 'income_score'], axis=1)
remaining_towns = remaining_towns.drop([11])
remaining_towns = remaining_towns.reset_index().drop('index', axis=1)
remaining_towns['cluster_labels'] = remaining_towns['cluster_labels'].astype(int)
print(remaining_towns.shape)
remaining_towns

(14, 6)


,borough,neighborhood,longitude,latitude,total_score,cluster_labels
0,Old City of Toronto,Waterfront Communities-The Island (77),-79.377202,43.633880,6.86,0
1,North York,Niagara (82),-79.412420,43.636681,3.53,0
2,North York,Willowdale East (51),-79.401484,43.770602,3.46,1
3,Old City of Toronto,Church-Yonge Corridor (75),-79.379017,43.659649,2.73,2
4,Scarborough,Islington-City Centre West (14),-79.543317,43.633463,2.63,1
5,Scarborough,Dovercourt-Wallace Emerson-Junction (93),-79.438541,43.665677,2.48,0
6,Old City of Toronto,Mount Pleasant West (104),-79.393360,43.704435,2.48,2
7,Old City of Toronto,Annex (95),-79.404001,43.671585,2.47,2
8,Scarborough,Woburn (137),-79.228586,43.766740,2.45,1
9,Scarborough,Mimico (includes Humber Bay Shores) (17),-79.500137,43.615924,2.27,1
